---
title: TRABAJO PRÁCTICO N° 1
subtitle: UN PRIMER ENCUENTRO CON LA EPH
abstract: El objetivo familiarización con la base de datos de la Encuesta Permanente de Hogares. Limpieza de datos, valores faltantes y análisis descriptivo. Medición de pobreza.

author: Martin Cargnel

---

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Parte 1: Familiarizandonos con la base EPH y limpieza

La Encuesta Permanente de Hogares (EPH) es un programa nacional de producción sistemática y permanente de indicadores sociales que lleva a cabo el Instituto Nacional de Estadística y Censos (INDEC), que permite conocer las características sociodemográficas y socioeconómicas de la población. Uno de los indicadores más valiosos sobre el mercado laboral que pueden obtenerse con los datos de esta encuesta es la tasa de desocupación.

## Breve introducción a la base

Utilizando información disponible en la página del INDEC, expliquen brevemente cómo se identifica a las personas pobres.


## Descargar los datos

Entren a la página https://www.indec.gob.ar/ y vayan a la sección Servicios y Herramientas → Bases de datos. Descarguen la base de microdatos de la Encuesta Permanente de Hogares (EPH) correspondiente al primer trimestre de 2005 y 2025 en formato .dta y .xls, respectivamente (una vez descargadas, las bases a usar deberán llamarse usu_individual_T105.dta y usu_individual_T125.xls). En la página web, también encontrará un diccionario de variables con el nombre de “Diseño de registro y estructura para las bases preliminares (hogares y personas)”. Descarguen el diccionario de cada año (los de 2005 y 2025). En estos archivos se les indica qué significa cada variable que aparece en la base de datos, en particular, en la sección de Diseño de registros de la base Personas.

In [181]:
#df_usu_hogar = pd.read_excel('data/usu_hogar_T125.xlsx')
df_individual_t105 = pd.read_stata('data/Individual_t105.dta')
df_individual_t125 = pd.read_excel('data/usu_individual_T125.xlsx')

A partir de ahora, cada grupo debe decidir trabajar con una región del país en específico y mantener dicha región en los próximos trabajos prácticos (ver variable REGION). Eliminen los datos de todas aquellas regiones que no se encuentren dentro de su región y unan ambos trimestres (2005 y 2025) en una sola base. 

In [182]:
df_individual_t125.columns = df_individual_t125.columns.str.lower()

In [183]:
df_individual_t105_filt = df_individual_t105[df_individual_t105['region'] == 'Gran Buenos Aires']

df_individual_t125_filt = df_individual_t125[df_individual_t125['region'] == 41]

In [184]:
df_total = pd.concat([df_individual_t105_filt, df_individual_t125_filt])
del df_individual_t125_filt, df_individual_t125, df_individual_t105_filt, df_individual_t105

Asegúrense de que todas las variables tengan el formato correcto. Seleccione 15 variables de interés (entre las cuales tienen que estar: CH04, CH06, CH07, CH08, NIVEL ED, ESTADO, CAT_INAC, IPCF) y reporten en una figura de heatmap la cantidad de valores faltantes para cada año (NA, o NaN en Python) en una tabla por cada año. Comenten qué variables de las 15 que seleccionaron tienen más valores faltantes y qué año.

In [185]:
variables_dadas = ['CH04', 'CH06', 'CH07', 'CH08', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'IPCF']

variables_elegidas = ['PP04A', 'PP05B2_ANO', 'pp07c','pp07g_59','PP07F4','pp07j','T_VI']

total_variables = variables_dadas + variables_elegidas
total_variables = [i.lower() for i in total_variables]
len(total_variables)

15

In [186]:
df_total = df_total[['ano4']+total_variables]

| Variable              | Descripción breve                                      |
|-----------------------|-------------------------------------------------------|
| ch04                  | Sexo                                                  |
| ch06                  | Edad                                                  |
| ch07                  | Estado civil                                          |
| ch08                  | Cobertura de salud                                    |
| nivel_ed              | Nivel educativo alcanzado                             |
| estado                | Condición de actividad                                |
| cat_inac              | Categoría de inactividad                             |
| ipcf                  | Ingreso per cápita familiar                           |
| pp04a                 | Rama de actividad principal                           |
| pp05b2_ano            | Años de antigüedad en la ocupación                    |
| pp07c                 | Empleo con fecha de finalización                                       |
| pp07g_59              | No tiene Vacaciones pagas, aguinaldo, dias pagos por enfermedad, obra social.                                    |
| PP07F4                   | No recibe ningun beneficio en el trabajo (vivienda, comida, etc.)                                |
|pp07j | Turno de trabajo |
| T_VI | Monto total de ingresos no laborables |

In [187]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14230 entries, 0 to 45413
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ano4        14230 non-null  float64
 1   ch04        14230 non-null  object 
 2   ch06        14230 non-null  object 
 3   ch07        14230 non-null  object 
 4   ch08        14230 non-null  object 
 5   nivel_ed    14230 non-null  object 
 6   estado      14230 non-null  object 
 7   cat_inac    14230 non-null  object 
 8   ipcf        14230 non-null  float64
 9   pp04a       11415 non-null  object 
 10  pp05b2_ano  11415 non-null  float64
 11  pp07c       11415 non-null  object 
 12  pp07g_59    11415 non-null  object 
 13  pp07f4      11415 non-null  object 
 14  pp07j       11415 non-null  object 
 15  t_vi        14230 non-null  float64
dtypes: float64(4), object(12)
memory usage: 1.8+ MB


In [188]:
df_total.groupby('ano4').apply(lambda x: x.isna().sum(), include_groups=False)

,ch04,ch06,ch07,ch08,nivel_ed,estado,cat_inac,ipcf,pp04a,pp05b2_ano,pp07c,pp07g_59,pp07f4,pp07j,t_vi
ano4,,,,,,,,,,,,,,,
2005.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025.0,0,0,0,0,0,0,0,0,2815,2815,2815,2815,2815,2815,0


Corregir variables si notan valores sin sentido (como ingresos negativos) de acuerdo a la documentación de la EPH (puede ser una codificación de no respuesta de los individuos) y eliminen estos valores extraños de sus 15 variables de interés. Comenten brevemente en el reporte dicho proceso de limpieza.

In [ ]:
df_total['ch04'] = df_total['ch04'].replace({'1': 'Varón', '2': 'Mujer'})

In [ ]:
df_total['ch04'] = df_total['ch04'].replace({'1': 'Varón', 1: 'Varón', '2': 'Mujer', 2: 'Mujer'})

In [ ]:
# Set negative or zero ages to 0 
df_total = df_total[df_total['ch06'] != 'Menos de 1 año']
df_total['ch06'] = df_total['ch06'].astype(int) 
#df_total = df_total.loc[df_total['ch06'] > 0]

In [ ]:
df_total['ch07'] = df_total['ch07'].replace({'1': 'Unido', '2': 'Casado', '3': 'Separado o divorciado',
                                             '4': 'Viudo', '5':'Soltero'})
#df_total = df_total[df_total['ch07']!='Ns./Nr.']

In [ ]:
df_total['ch08'].unique()

In [165]:
df_total_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14026 entries, 0 to 45413
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ano4        14026 non-null  float64
 1   ch04        14026 non-null  object 
 2   ch06        14026 non-null  int64  
 3   ch07        14026 non-null  object 
 4   ch08        14019 non-null  object 
 5   nivel_ed    14026 non-null  object 
 6   estado      14026 non-null  object 
 7   cat_inac    14026 non-null  object 
 8   ipcf        14026 non-null  float64
 9   pp04a       11240 non-null  object 
 10  pp05b2_ano  11249 non-null  float64
 11  pp07c       11026 non-null  object 
 12  pp07g_59    11237 non-null  object 
 13  pp07f4      11237 non-null  object 
 14  pp07j       11236 non-null  object 
 15  t_vi        14026 non-null  float64
dtypes: float64(4), int64(1), object(11)
memory usage: 1.8+ MB


# Parte II: Primer Análisis Exploratorio

## Realicen un gráfico de barras mostrando la composición por sexo para 2005 y 2025 en su región. Comenten los resultados.

## Matriz correlación

Realicen una matriz de correlación para 2005 y 2025 con las siguientes variables: CH04, CH06, CH07, CH08, NIVEL ED, ESTADO, CAT_INAC, IPCF. Crear las variables dicotómicas binarias necesarias (variables dummies) y renombrar dichas variables para que las etiquetas tengan sentido en el gráfico de correlación. Utilicen alguno de los comandos disponibles en este link para graficar la matriz de correlación. Comenten los resultados. 


# Parte III: Conociendo a los pobres y no pobres 

Los siguientes incisos apuntan a ver los resultados para su región seleccionada y comparando 2005 con 2025.

Uno de los grandes problemas de la EPH es la creciente cantidad de hogares que no reportan sus ingresos (ver por ejemplo el siguiente informe). ¿Cuántas personas no respondieron cuál es su condición de actividad? Guarden como una base distinta llamada respondieron las observaciones donde respondieron la pregunta sobre su ingreso total familiar (ITF). Las observaciones con ITF=0 guárdenlas en una base bajo el nombre norespondieron.

Utilizando el archivo tabla_adulto_equiv.xlsx, agreguen a su base de datos una columna llamada adulto_equiv que contenga los valores de adulto equivalente de cada persona según su sexo y edad (por ejemplo, a un varón de 2 años le corresponde 0.46). Finalmente, con el comando groupby sumen esta nueva columna para las personas que pertenecen a un mismo hogar y guarden ese dato en una columna llamada ad_equiv_hogar .


Sabiendo que la Canasta Básica Total para un adulto equivalente en el primer trimestre de 2025 es aproximadamente $365.177, agreguen a la base respondieron una columna llamada ingreso_necesario que sea el producto de este valor por ad_equiv_hogar. Para el primer trimestre de 2005 la Canasta Básica Total para un adulto equivalente era aproximadamente $205,07. Note que este es el valor mínimo que necesita ese hogar para no ser pobre.


Por último, agreguen a respondieron una columna llamada pobre que tome valor 1 si el ITF es menor al ingreso necesario que necesita esa familia, y 0 en caso contrario. ¿Cuántos pobres identificaron para cada año? ¿Qué porcentaje de la muestra representa?

(Opcional) Muestren estadísticas descriptivas relevantes de pobre en una tabla, comparando 2005 con 2025. Además, hagan 2 gráficos exploratorios a elección usando la variable pobre. Comenten. tulo
